# Importar librerias

Importar librerias necesarias para manipular datos, realizar gráficos, hacer web scraping sobre el stock screener de las acciones en las que deseamos invertir y por supuesto, la libreria necesaria para formular el problema de optimización convexa y resolverlo.

In [3]:
import pandas_datareader.data as web
import datetime as dt
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import numpy as np
import scipy.sparse as sp

from bs4 import BeautifulSoup
from yahoo_fin.stock_info import get_data

# Web scraping y creación de conjunto de datos

En esta sección se realizo web scraping para extraer los tickers de las empresas en las que deseamos invertir. Estas empresas se filtraron de manera que estuvieran en el sector de tecnología y tuvieran una capitalización bursátil grande (más de USD $10B).

Únicamente se tuvieron en cuenta las empresas que tenian datos desde 2011-7-2 hasta 2021-7-2, quedando asi con 103 tickers de los 195 que había originalmente en el stock screener: [Yahoo Finance Stock Screener](https://tinyurl.com/StockScreenerYF).

In [4]:
tickers = []

with open('yahoofinance1.html', 'r',  encoding="utf8") as html_file:
    content = html_file.read()
    soup = BeautifulSoup(content, 'lxml')
    ticker_tags = soup.find_all('a', attrs={'data-test' : 'quoteLink'})
    for tag in ticker_tags:
        tickers.append(tag.text)

with open('yahoofinance2.html', 'r',  encoding="utf8") as html_file:
    content = html_file.read()
    soup = BeautifulSoup(content, 'lxml')
    ticker_tags = soup.find_all('a', attrs={'data-test' : 'quoteLink'})
    for tag in ticker_tags:
        tickers.append(tag.text)   
        
tickers = ['SPYG', 'QQQ', 'VUG', 'IYW', 'VOO', 'VTI', 'SCHG', 'VGT']

start = dt.datetime(2013, 1, 1)
end = dt.datetime(2021, 7, 13)

dataset = pd.DataFrame()

for ticker in tickers:
    data = get_data(ticker, start_date=start, end_date=end)
    data[ticker] = data['adjclose']
    
    if dataset.empty:
        dataset = data[[ticker]]
    else:
        dataset = dataset.join(data[[ticker]], how = 'outer')

dataset = dataset.dropna(axis='columns')

# Análisis exploratorio del conjunto de datos

En esta sección se van a observar ciertas características del conjunto de datos.

A continuación mostramos una previsualización de como luce el conjunto de datos:

In [ ]:
dataset

In [ ]:
print("Número de columnas:", len(dataset.columns))

Acá observamos un ejemplo de como es la visualización de una de las acciones que vamos a incluir en el portafolio. Esta acción 'FIS' corresponde a la primera que sale en el conjunto de datos anterior. Se puede obtener una visualización igual para cada una de las columnas del conjunto de datos que tenemos, en este caso estamos graficando la primera columna que corresponde a la acción 'FIS'.

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(dataset.iloc[:, 3])
plt.title("Precio histórico de {}".format(dataset.columns[3]))
plt.show()

# Formulación del problema

A continuación vamos a definir que significa cada variable del problema de optimización a resolver:

- $n :$ numero de activos en el portafolio.
- $T :$ periodo de tiempo durante el cual compro/vendo los activos de mi portafolio.
- $p_i = \frac{s_1^{(i)}}{s_0^{(i)}} :$ cambio de precio relativo del activo $i$ durante el periodo $T$, donde $s_0^{(i)}$ es el precio inicial y $s_1^{(i)}$ es el precio al final del periodo $T$.
- $w \in \textbf{R}^n :$ vector que tiene los pesos que se le asignan a cada activo del portafolio.
  Se le asigna el peso $w_i$ al activo $i$ para $i=1, 2, ..., n$.
- $x_i :$ cantidad de acciones compradas/vendidas del activo $i$.
- $\sum_{i=1}^n x_i :$ capital total disponible.
- $r_i=p_ix_i :$ retorno del activo $i$.
- $r = p^Tx :$ retorno total de portafolio.
- $\overline{r} = \overline{p}^Tx :$ media del vector $r$ (promedio de los retornos).
- $E(p) = \overline{p} :$ media del vector $p$
- $E((p-\overline{p})(p-\overline{p})^T) = \Sigma :$ matriz de varianzas y covarianzas de los cambios de precio relativos de los activos.
- $\sigma_r^2 = x^T\Sigma x :$ varianza de los retornos del portafolio (riesgo de los retornos).
- Todos los pesos del portafolio deben sumar uno, esto es $\textbf{1}^Tw=1$.


Para este problema básicamente buscamos dos objetivos: minimizar el riesgo y maximizar las ganancias del portafolio, esto lo logramos asignando los pesos del vector $w$ de manera estratégica teniendo en cuenta que los retornos de cada activo no son los mismos, es decir hay activos que a la fecha han tenido un retorno mayor o menor respecto a otros y con caídas de mayor o menor porcentaje respecto a otros activos. 

$$
\begin{array}{cr}
\mbox{minimizar}     & w^T\Sigma w \\
\mbox{sujeto a}     & \textbf{1}^Tw=1, \\
& \overline{p}^Tw \geq r_{\text{min}}, \\
& w \geq 0\\
\end{array} \\
$$

donde $w^T\Sigma w$ es el riesgo del portafolio.

# Solución del problema

Para solucionar este problema podemos utilizar el **método de Lagrange** visto en clase en el capítulo de Dualidad, entonces definimos el **Lagrangiano** como:

$$
L(w, \lambda, \nu) = w^T\Sigma w - \lambda w - \nu(\textbf{1}^Tw-1)
$$

Por lo tanto la **función dual de Lagrange** queda definida como:

$$
g(\lambda, \nu)=\text{inf}_wL(w, \lambda, \nu)
$$

Para hallar este ínfimo, calculamos el vector gradiente de $L(w, \lambda, \nu)$ y lo igualamos a $0$:

$$
\frac{\partial L}{\partial w} = 2\gamma\Sigma w - \lambda - \nu\textbf{1} = 0, \\
\implies \Sigma w = \frac{1}{2\gamma}(\lambda + \nu\textbf{1})
$$

Si asumimos que la matriz $\Sigma$ es definida positiva (esto es, que los activos en el portafolio **no** están perfectamente correlacionados), entonces:

$$
w^*=\frac{1}{2\gamma}\Sigma^{-1}(\lambda + \nu\textbf{1})
$$

# Solución computacional

En este caso vamos a utilizar al paquete <code>cvxpy</code> para solucionar el problema de optimización convexo.

In [5]:
import cvxpy as cp

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

Recordemos que ya tenemos nuestro conjunto de datos <code>dataset</code>:

In [ ]:
dataset

Creamos el conjunto de datos <code>returns</code> a partir de nuestro conjunto de datos original.

In [6]:
returns = dataset/dataset.shift(1)
returns.dropna(inplace=True)
returns = np.log(returns)
returns

,SPYG,QQQ,VUG,IYW,VOO,VTI,SCHG,VGT
2013-01-03,-0.003427,-0.005221,-0.002879,-0.007293,-0.000899,-0.001465,-0.001712,-0.005913
2013-01-04,0.002980,-0.003297,0.003289,-0.007068,0.004187,0.005052,0.003421,-0.004387
2013-01-07,-0.002234,0.000301,-0.001918,-0.001809,-0.002689,-0.002656,-0.001709,-0.001561
2013-01-08,-0.001193,-0.001952,-0.002058,-0.003350,-0.002997,-0.002663,-0.001426,-0.002560
2013-01-09,0.001939,0.003751,0.003703,0.002514,0.002997,0.003062,0.004274,0.003412
...,...,...,...,...,...,...,...,...
2021-07-06,0.005453,0.004313,0.006245,0.005147,-0.001883,-0.002178,0.007393,0.004942
2021-07-07,0.004650,0.002108,0.001914,0.002169,0.003436,0.001334,0.003743,0.003838
2021-07-08,-0.006672,-0.006058,-0.007196,-0.009601,-0.007994,-0.008477,-0.007903,-0.009796
2021-07-09,0.006518,0.006224,0.007776,0.008319,0.010495,0.012025,0.007502,0.009403


Ahora definimos el vector $\overline{p}$ y la matriz de varianzas y covarianzas $\Sigma:$

In [7]:
pBar = np.asarray(np.mean(returns.to_numpy().T, axis=1))
Sigma = np.asmatrix(np.cov(returns.to_numpy().T))
rMin = 0.0008 # Debe ser menor a np.max(pBar)

$$
\begin{array}{cr}
\mbox{minimizar}     & w^T\Sigma w \\
\mbox{sujeto a}     & \textbf{1}^Tw=1, \\
& \overline{p}^Tw \geq r_{\text{min}}, \\
& w \geq 0\\
\end{array} \\
$$
Entonces, calculamos el punto óptimo de nuestro problema de optimización con la libreria <code>cvxpy</code> teniendo en cuenta las restricciones de que los pesos deben sumar 1 y queremos un portafolio en el que **unicamente compramos**: 

In [8]:
n = len(returns.columns)

w = cp.Variable(n)
risk = cp.quad_form(w, Sigma)

constraints = [pBar.T @ w >= rMin, cp.sum(w) == 1, w >= 0]
prob = cp.Problem(cp.Minimize(risk), 
                  constraints)

In [9]:
prob.solve()

0.00014752723920421687

In [10]:
prob.status

'optimal'

Habiendo hallado el vector de pesos que minimiza el riesgo de nuestro portafolio, si suponemos que tenemos un capital total disponible de **US$5.000**, entonces el vector que nos indica el valor que debemos comprar de cada empresa es:

In [12]:
i = 0
for column in returns.columns:
    if np.round(w.value[i]*5000) > 1:
        print("Inversión para {}: ${}".format(column, np.round(w.value[i]*320)))
    i += 1

Inversión para SPYG: $36.0
Inversión para QQQ: $259.0
Inversión para SCHG: $24.0
